<h1>Installing RDKit</h1>

In [203]:
import rdkit
import deepchem as dc

In [204]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn

In [205]:
df = pd.read_excel('/Users/misspotato/Documents/Github/Binding-Free-Energy-Prediction-Host-Guest-System/Results/host-guest-dataset_clean.xlsx')
df = df.dropna()

In [206]:
# Shuffle the data randomly
np.random.shuffle(df.values)

In [207]:
df.sample(frac = 1).reset_index(drop = True)

,ID,dataset group name,Dataset Name,Host,Guest,Ex _G_(kcal/mol),Ex _G_SEM,EX _H_(kcal/mol),EX _H_SEM,pb_guest_Etot,...,gb_host_1-4EEL,gb_host_EELEC,gb_host_EGB,gb_host_ESURF,gb_delta_H,pb_delta_H,EX _delta_H_(kcal/mol),gb_Ex_difference,SQR_gbnsr6_Ex_difference,pb_Ex_difference
0,11,Mobley benchmarkset,cd-set1,acd,guest-11,-2.720,0.004,-3.28,0.02,-51.7020,...,422.5394,-418.6250,-97.5622,5.1074,-6.8615,-5.0642,-3.28,-3.5815,12.827142,-1.7842
1,17,Mobley benchmarkset,cd-set1,acd,guest-17,-3.227,1.135,-0.93,0.32,-5.0152,...,423.2458,-437.2679,-91.6246,5.0761,-11.4839,-11.8241,-0.93,-10.5539,111.384805,-10.8941
2,36,Mobley benchmarkset,cd-set2,bcd,guest-14,-1.554,0.167,0.88,0.17,0.1436,...,497.2683,-495.0822,-121.3069,5.9335,-4.8565,-5.9462,0.88,-5.7365,32.907432,-6.8262
3,32,Mobley benchmarkset,cd-set2,bcd,guest-10,-2.192,0.013,-2.89,0.05,-47.3442,...,488.4410,-486.0020,-117.8558,5.9297,-11.7835,-11.7177,-2.89,-8.8935,79.094342,-8.8277
4,37,Mobley benchmarkset,cd-set2,bcd,guest-15,-4.175,0.010,-2.29,0.03,-6.1893,...,500.9283,-493.6747,-122.1523,5.9483,-13.8003,-12.9928,-2.29,-11.5103,132.487006,-10.7028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,8,Mobley benchmarkset,cd-set1,acd,guest-8,-4.622,0.017,-4.89,0.03,-70.0470,...,418.4633,-421.1437,-95.0137,5.1211,-15.8181,-14.5525,-4.89,-10.9281,119.423370,-9.6625
58,56,SAMPL5,sample,OAH,guest-3,-5.060,0.010,-5.93,0.08,46.0430,...,-82.4983,678.2864,-1388.0243,8.3998,-21.2137,-17.7786,-5.93,-15.2837,233.591486,-11.8486
59,61,SAMPL5,sample,OAME,guest-2,-5.040,0.030,-7.56,0.10,-88.3150,...,350.2523,385.2185,-1350.3593,8.6313,-2.3246,-0.0453,-7.56,5.2354,27.409413,7.5147
60,55,SAMPL5,sample,OAH,guest-2,-4.250,0.010,-4.39,0.03,-89.4726,...,-82.6409,675.4077,-1390.4029,8.3838,-9.2216,-7.4032,-4.39,-4.8316,23.344359,-3.0132


<h1>Reading Mobley PDB files</h1>
<p>Here each PDB file will be read and saved in Mol data type defined in RDKit and used by DeepChem</p>

In [208]:
# Dictionary with complex names as keys and molecule as values
PDBs = {}
from os import listdir
from os.path import isfile, join
mypath = '/Users/misspotato/Documents/Github/Binding-Free-Energy-Prediction-Host-Guest-System/PDB'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for f in onlyfiles:
    PDBs.update({f.split('.')[0].replace('-s', '') : rdkit.Chem.rdmolfiles.MolFromPDBFile(mypath + '/' + f)})

In [209]:
PDBs.pop('',None)

<h1>Featurizing</h1>
<p>GraphConv model needs ConvMolFeaturizer</p>

In [210]:
featurizer = dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

In [211]:
X = []
X_ids = []
one_add = 0 if len(PDBs.keys()) % 2 == 0 else 1
for k in PDBs.keys():
    X_ids.append(k)
    X.append(featurizer.featurize(PDBs[k]))

In [212]:
X = []
X_ids = []
one_add = 0 if len(PDBs.keys()) % 2 == 0 else 1
for k in PDBs.keys():
    X_ids.append(k)
    X.append(featurizer.featurize(PDBs[k]))
X = [x[0] for x in X]
X_train_featurized = X[:int(len(X) / 2)]
X_test_featurized = X[int(len(X) / 2) + one_add:]

In [213]:
host_names = [i.split('-')[0] for i in X_ids]
guest_names = ['guest-' + (i.split('-')[1].replace('s', '')) for i in X_ids]

In [214]:
host_names_train = host_names[:int(len(host_names) / 2)]
guest_names_train = guest_names[:int(len(guest_names) / 2)]
host_names_test = host_names[int(len(host_names) / 2) + one_add:]
guest_names_test = guest_names[int(len(guest_names) / 2) + one_add:]

In [215]:
x_add_train, x_add_test, y_train, y_test = [], [], [], []
for i in range(len(host_names_train)):
    new_df = df[(df['Host'] == host_names_train[i]) & (df['Guest'] == guest_names_train[i])]
    y_train.append(new_df['EX _H_(kcal/mol)'].to_numpy()[0])
    x_add_train.append(new_df[[c for c in df.columns if ('Etot' not in c) and ('delta' not in c)
                         and ('Ex_difference' not in c) and ('gb_' in c or 'VDWAALS' in c)]].to_numpy()[0])
y_train = np.array(y_train)
    
for i in range(len(host_names_test)):
    new_df = df[(df['Host'] == host_names_test[i]) & (df['Guest'] == guest_names_test[i])]
    y_test.append(new_df['EX _H_(kcal/mol)'].to_numpy()[0])
    x_add_test.append(new_df[[c for c in df.columns if ('Etot' not in c) and ('delta' not in c)
                         and ('Ex_difference' not in c) and ('gb_' in c or 'VDWAALS' in c)]].to_numpy()[0])
y_test = np.array(y_test)

In [216]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol

x_preprocessed_train, x_preprocessed_test = [], []

## for X train
multiConvMol = ConvMol.agglomerate_mols(X_train_featurized)
x_preprocessed_train = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
    x_preprocessed_train.append(multiConvMol.get_deg_adjacency_lists()[i])
x_preprocessed_train.append(np.array(x_add_train))

## for X test
multiConvMol = ConvMol.agglomerate_mols(X_test_featurized)
x_preprocessed_test = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
    x_preprocessed_test.append(multiConvMol.get_deg_adjacency_lists()[i])
x_preprocessed_test.append(np.array(x_add_test))

In [217]:
x_train = np.full([14, np.max([v.shape[0] for v in x_preprocessed_train]),
                  np.max([v.shape[1] for v in x_preprocessed_train if len(v.shape) > 1])], 1.123456)
for i,j in enumerate(x_preprocessed_train):
    if len(j.shape) > 1:
        x_train[i][:j.shape[0],:j.shape[1]] = np.array(j)
    else:
        x_train[i][:len(j), :1] = np.array(j).reshape(j.shape[0], 1)
x_train = x_train.reshape([1] + list(x_train.shape))

x_test = np.full([14, np.max([v.shape[0] for v in x_preprocessed_test]),
                  np.max([v.shape[1] for v in x_preprocessed_test if len(v.shape) > 1])], 1.123456)
for i,j in enumerate(x_preprocessed_test):
    if len(j.shape) > 1:
        x_test[i][:j.shape[0],:j.shape[1]] = np.array(j)
    else:
        x_test[i][:len(j), :1] = np.array(j).reshape(j.shape[0], 1)
x_test = x_test.reshape([1] + list(x_test.shape))

In [218]:
x_train.shape

(1, 14, 2833, 75)

In [219]:
x_test.shape

(1, 14, 3251, 75)

<h1>Creating Model</h1>

In [220]:
# X_features = []
# for x_feat in X:
#     multiConvMol = ConvMol.agglomerate_mols([x_feat])
#     new_x_feat = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
#     for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
#         new_x_feat.append(multiConvMol.get_deg_adjacency_lists()[i])
#     X_features.append(new_x_feat)

In [221]:
## !!!!!!!! important
## !!!!!!!! important
## !!!!!!!! important
## !!!!!!!! important
input_shapes = [i.shape for i in x_preprocessed_train]

In [222]:
input_shapes

[(2833, 75),
 (11, 2),
 (2833,),
 (649, 1),
 (981, 2),
 (1203, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (31, 15)]

In [223]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Concatenate
from tensorflow.keras import initializers

batch_size = int(len(df) / 2)

class GBGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(GBGraphConvModel, self).__init__()
    self.gc1 = GraphConv(64, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(64, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(128, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(1)
    self.dense3 = layers.Dense(1, 
         kernel_initializer=initializers.Constant([.5, -1, -1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1]),
         bias_initializer=initializers.Zeros())

  def call(self, inputs):
    inputs = inputs[0]
    x = []
#     input_shapes = [[4822, 75], [11, 2], [4822], [1142, 1], [1635, 2], [2042, 3],
#                    [3, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10]]
    for i in range(len(input_shapes)):
        x.append(tf.reshape(inputs[i][inputs[i] != 1.123456], input_shapes[i]))
    for i in range(1, len(input_shapes)):
        x[i] = tf.cast(x[i], tf.int32)
    x_add = tf.reshape(inputs[13][inputs[13] != 1.123456], [batch_size, 15])
    gc1_output = self.gc1(x)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + x[1:])

    gc2_output = self.gc2([gp1_output] + x[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + x[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + x[1:])

    model_var = self.dense2(readout_output)
    binding_affinity = tf.concat([model_var, x_add], axis=1)
    return self.dense3(binding_affinity)
model = GBGraphConvModel()
model.compile(loss='mse', optimizer='adam')

In [224]:
train_history = model.fit(x_train, y_train.reshape([1, -1]), epochs=500)

Epoch 1/500


/Users/misspotato/opt/miniconda3/envs/rdkit-deepchem-jupyter/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/gb_graph_conv_model_4/graph_pool_9/Reshape_11:0", shape=(649,), dtype=int32), values=Tensor("gradient_tape/gb_graph_conv_model_4/graph_pool_9/Reshape_10:0", shape=(649, 64), dtype=float32), dense_shape=Tensor("gradient_tape/gb_graph_conv_model_4/graph_pool_9/Cast_3:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/Users/misspotato/opt/miniconda3/envs/rdkit-deepchem-jupyter/lib/python3.9/site-packages/tensorflow/python/framework/indexed_slices.py:447: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/gb_graph_conv_model_4/graph_pool_9/Reshape_14:0", shape=(1962,), dtype=int32), values=Tensor("gradient_tape/gb_graph_conv_model_4/graph_pool_9/Reshap

1/1 [==============================] - 5s 5s/step - loss: 78.2213
Epoch 2/500
1/1 [==============================] - 0s 43ms/step - loss: 49.6618
Epoch 3/500
1/1 [==============================] - 0s 52ms/step - loss: 32.5092
Epoch 4/500
1/1 [==============================] - 0s 45ms/step - loss: 24.3505
Epoch 5/500
1/1 [==============================] - 0s 43ms/step - loss: 22.8390
Epoch 6/500
1/1 [==============================] - 0s 47ms/step - loss: 25.4951
Epoch 7/500
1/1 [==============================] - 0s 41ms/step - loss: 29.8017
Epoch 8/500
1/1 [==============================] - 0s 40ms/step - loss: 32.4366
Epoch 9/500
1/1 [==============================] - 0s 46ms/step - loss: 32.4312
Epoch 10/500
1/1 [==============================] - 0s 52ms/step - loss: 30.6171
Epoch 11/500
1/1 [==============================] - 0s 45ms/step - loss: 27.9032
Epoch 12/500
1/1 [==============================] - 0s 47ms/step - loss: 24.4089
Epoch 13/500
1/1 [==============================] -

1/1 [==============================] - 0s 41ms/step - loss: 10.8133
Epoch 103/500
1/1 [==============================] - 0s 38ms/step - loss: 10.7420
Epoch 104/500
1/1 [==============================] - 0s 41ms/step - loss: 10.6714
Epoch 105/500
1/1 [==============================] - 0s 45ms/step - loss: 10.6095
Epoch 106/500
1/1 [==============================] - 0s 42ms/step - loss: 10.5545
Epoch 107/500
1/1 [==============================] - 0s 42ms/step - loss: 10.4960
Epoch 108/500
1/1 [==============================] - 0s 46ms/step - loss: 10.4491
Epoch 109/500
1/1 [==============================] - 0s 47ms/step - loss: 10.4186
Epoch 110/500
1/1 [==============================] - 0s 44ms/step - loss: 10.4217
Epoch 111/500
1/1 [==============================] - 0s 38ms/step - loss: 10.4036
Epoch 112/500
1/1 [==============================] - 0s 44ms/step - loss: 10.3828
Epoch 113/500
1/1 [==============================] - 0s 40ms/step - loss: 10.3487
Epoch 114/500
1/1 [===========

1/1 [==============================] - 0s 42ms/step - loss: 8.7216
Epoch 204/500
1/1 [==============================] - 0s 43ms/step - loss: 8.6845
Epoch 205/500
1/1 [==============================] - 0s 42ms/step - loss: 8.7265
Epoch 206/500
1/1 [==============================] - 0s 45ms/step - loss: 8.6706
Epoch 207/500
1/1 [==============================] - 0s 47ms/step - loss: 8.6926
Epoch 208/500
1/1 [==============================] - 0s 46ms/step - loss: 8.7234
Epoch 209/500
1/1 [==============================] - 0s 42ms/step - loss: 8.7044
Epoch 210/500
1/1 [==============================] - 0s 41ms/step - loss: 8.6743
Epoch 211/500
1/1 [==============================] - 0s 41ms/step - loss: 8.6576
Epoch 212/500
1/1 [==============================] - 0s 44ms/step - loss: 8.6421
Epoch 213/500
1/1 [==============================] - 0s 45ms/step - loss: 8.6224
Epoch 214/500
1/1 [==============================] - 0s 47ms/step - loss: 8.6345
Epoch 215/500
1/1 [=======================

1/1 [==============================] - 0s 47ms/step - loss: 8.2202
Epoch 305/500
1/1 [==============================] - 0s 43ms/step - loss: 8.2183
Epoch 306/500
1/1 [==============================] - 0s 41ms/step - loss: 8.2164
Epoch 307/500
1/1 [==============================] - 0s 44ms/step - loss: 8.2147
Epoch 308/500
1/1 [==============================] - 0s 43ms/step - loss: 8.2127
Epoch 309/500
1/1 [==============================] - 0s 45ms/step - loss: 8.2109
Epoch 310/500
1/1 [==============================] - 0s 45ms/step - loss: 8.2090
Epoch 311/500
1/1 [==============================] - 0s 48ms/step - loss: 8.2074
Epoch 312/500
1/1 [==============================] - 0s 45ms/step - loss: 8.2057
Epoch 313/500
1/1 [==============================] - 0s 51ms/step - loss: 8.2041
Epoch 314/500
1/1 [==============================] - 0s 53ms/step - loss: 8.2025
Epoch 315/500
1/1 [==============================] - 0s 46ms/step - loss: 8.2009
Epoch 316/500
1/1 [=======================

1/1 [==============================] - 0s 48ms/step - loss: 8.0901
Epoch 406/500
1/1 [==============================] - 0s 49ms/step - loss: 8.0891
Epoch 407/500
1/1 [==============================] - 0s 49ms/step - loss: 8.0883
Epoch 408/500
1/1 [==============================] - 0s 55ms/step - loss: 8.0874
Epoch 409/500
1/1 [==============================] - 0s 46ms/step - loss: 8.0865
Epoch 410/500
1/1 [==============================] - 0s 50ms/step - loss: 8.0857
Epoch 411/500
1/1 [==============================] - 0s 46ms/step - loss: 8.0850
Epoch 412/500
1/1 [==============================] - 0s 46ms/step - loss: 8.0843
Epoch 413/500
1/1 [==============================] - 0s 44ms/step - loss: 8.0835
Epoch 414/500
1/1 [==============================] - 0s 52ms/step - loss: 8.0827
Epoch 415/500
1/1 [==============================] - 0s 46ms/step - loss: 8.0820
Epoch 416/500
1/1 [==============================] - 0s 46ms/step - loss: 8.0812
Epoch 417/500
1/1 [=======================

In [225]:
input_shapes = [i.shape for i in x_preprocessed_test]

In [226]:
input_shapes

[(3251, 75),
 (11, 2),
 (3251,),
 (679, 1),
 (1189, 2),
 (1377, 3),
 (6, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (31, 15)]

In [227]:
test_history = model.evaluate(x_test, y_test.reshape([1, -1]))

1/1 [==============================] - 1s 1s/step - loss: 34.6240


In [228]:
np.sqrt(34.6240)

5.884216175498653

<h1> Physics based model RMSE </h1>

In [ ]:
import math
train_sum=0
for i in range(len(host_names_train)):
    new_df = df[(df['Host'] == host_names_train[i]) & (df['Guest'] == guest_names_train[i])]
    train_sum += new_df['gb_Ex_difference'].to_numpy()[0] **2


test_sum = 0
for i in range(len(host_names_test)):
    new_df = df[(df['Host'] == host_names_test[i]) & (df['Guest'] == guest_names_test[i])]
    test_sum += new_df['gb_Ex_difference'].to_numpy()[0] **2
    

In [ ]:
physics_based_rmse_train = math.sqrt(train_sum / len(host_names_train))

In [ ]:
physics_based_rmse_test = math.sqrt((test_sum) / len(host_names_test))

In [ ]:
print(f"RMSE on training set is : {physics_based_rmse_train}")
print(f"rmse on testing set is : {physics_based_rmse_test}")
# Total rmse
total_rmse_physics = np.sqrt(np.mean((df['EX _H_(kcal/mol)'].to_numpy() - df['gb_delta_H'].to_numpy())**2))
print(f"RMSE of the total data: {total_rmse_physics}")

<h1> Loss per epoch </h1>

In [173]:
print(train_history)

<h1> Cross Validation </h1>

In [236]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score